# Tutorial 1.6: Framework Integrations

## Working with Multiple GenAI Frameworks

Welcome to framework integrations! MLflow supports 30+ GenAI frameworks, making it the most flexible platform for LLM development. This notebook shows you how to work with the most popular frameworks.

### What You'll Learn
- Overview of MLflow's framework integrations
- Working with OpenAI (direct API)
- Working with LangChain (chains and agents)
- Working with LlamaIndex (RAG and indexing)
- Comparing frameworks and choosing the right one
- Best practices for each framework

### Prerequisites
- Completed previous notebooks (1.1-1.5)
- OpenAI API key configured

### Estimated Time: 10-15 minutes

---
## Step 1: Framework Overview

### MLflow's 30+ Integrations

MLflow provides automatic tracing for:

**LLM Providers:**
- OpenAI, Anthropic, Cohere, Azure OpenAI
- AWS Bedrock, Google Vertex AI
- Ollama, vLLM, Together AI

**Frameworks:**
- LangChain, LlamaIndex, Haystack
- DSPy, AutoGen, CrewAI
- Guardrails AI, Phoenix

### Comparison Matrix

| Feature | OpenAI | LangChain | LlamaIndex |
|---------|--------|-----------|------------|
| **Complexity** | Low | Medium | Medium |
| **Learning Curve** | Easy | Medium | Medium |
| **Use Case** | Direct calls | Workflows | Doc Q&A |
| **Tracing** | ✅ Auto | ✅ Auto | ✅ Auto |
| **Agents** | Manual | ✅ Built-in | ✅ Built-in |
| **RAG** | Manual | ✅ Built-in | ✅ Built-in |
| **Customization** | Full | High | High |
| **Performance** | Fast | Medium | Medium |

### When to Use Each

**OpenAI Direct API:**
- Simple Q&A applications
- Maximum control over prompts
- Lowest latency
- Custom implementations

**LangChain:**
- Complex multi-step workflows
- Agent applications with tools
- Need for abstractions
- Rapid prototyping

**LlamaIndex:**
- Document-heavy applications
- Advanced indexing strategies
- Multiple data sources
- Knowledge management

---
## Step 2: Environment Setup

In [ ]:
# Install additional frameworks
!uv add langchain langchain-openai llama-index

print("✅ Frameworks installed")

In [ ]:
import mlflow
from dotenv import load_dotenv
from utils.clnt_utils import is_databricks_client, get_databricks_client, get_openai_client

# Load environment
load_dotenv()

# Configure MLflow
mlflow.set_tracking_uri("http://localhost:5000")

use_databricks_provider = is_databricks_client()
if use_databricks_provider:
    client = get_databricks_client()
else:
    # Initialize OpenAI
    client = get_openai_client()

model_name = "databricks-gpt-5-mini" if use_databricks_provider else "gpt-5-mini"

print("✅ Environment configured: using", "Databricks" if use_databricks_provider else "OpenAI", "client")
print(f"   MLflow version: {mlflow.__version__}")
print(f"   Tracking URI: {mlflow.get_tracking_uri()}")
print(f"   Model name: {model_name}")

#Enable OpenAI autologging
mlflow.openai.autolog()

mlflow.set_experiment("09-framework-integrations")

print("✅ Environment configured")
print("   OpenAI autologging: ENABLED")

print(f"   MLflow version: {mlflow.__version__}")

---
## Step 3: OpenAI Framework (Already Covered)

We've been using OpenAI throughout this tutorial series.

### Quick Review

### OpenAI Strengths

- ✅ **Simplicity**: Direct API calls
- ✅ **Performance**: No abstraction overhead
- ✅ **Control**: Full control over prompts
- ✅ **Latest features**: Immediate access to new models
- ✅ **Documentation**: Extensive official docs

---
## Step 4: LangChain Framework

LangChain provides abstractions for building LLM applications.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Enable LangChain autologging
mlflow.langchain.autolog()

print("✅ LangChain autologging enabled")

In [ ]:
from utils.clnt_utils import get_databricks_langchain_chat_client, get_langchain_chat_openai_client
# Simple LangChain chain
print("\n🔗 LangChain Example 1: Simple Chain\n")

# Create prompt template
prompt = ChatPromptTemplate.from_template(
    "You are a {role}. Answer: {question}"
)

# Create LLM based on provider

if use_databricks_provider:
    llm = get_databricks_langchain_chat_client(model_name, temperature=1.0)
else:
    llm = get_langchain_chat_openai_client(mode=model_name, temperature=1.0)

# Create chain using LCEL (LangChain Expression Language)
chain = prompt | llm | StrOutputParser()

# Run chain (automatically traced!)
result = chain.invoke({
    "role": "MLflow expert",
    "question": "What makes LangChain different from using OpenAI directly?"
})

print(result)
print("\n✅ Chain execution fully traced!")
print("   - Prompt construction")
print("   - LLM call")
print("   - Output parsing")

In [ ]:
# More complex chain with multiple steps
print("\n🔗 LangChain Example 2: Multi-Step Chain\n")


# Step 1: Generate topic
topic_prompt = ChatPromptTemplate.from_template(
    "Generate a technical topic about {domain}"
)
topic_chain = topic_prompt | llm | StrOutputParser()

# Step 2: Create outline
outline_prompt = ChatPromptTemplate.from_template(
    "Create a 3-point outline for: {topic}"
)
outline_chain = outline_prompt | llm | StrOutputParser()

# Execute pipeline
topic = topic_chain.invoke({"domain": "MLOps"})
print(f"Topic: {topic}\n")

outline = outline_chain.invoke({"topic": topic})
print(f"Outline:\n{outline}")

print("\n✅ Multi-step chain traced!")
print("   Each chain creates separate spans")
print("   Full execution visible in MLflow UI")

### LangChain Strengths

- ✅ **Abstractions**: Reusable components
- ✅ **Chains**: Complex multi-step workflows
- ✅ **Agents**: Built-in agent patterns
- ✅ **Tools**: Easy tool integration
- ✅ **Community**: Large ecosystem
- ✅ **RAG**: Built-in retrieval components

---
## Step 5: LlamaIndex Framework

LlamaIndex specializes in document indexing and retrieval.

In [ ]:
from llama_index.core import VectorStoreIndex, Document

# Enable LlamaIndex autologging
mlflow.llama_index.autolog()

print("✅ LlamaIndex autologging enabled")

In [ ]:
# Create sample documents
print("\n📚 LlamaIndex Example: Document Q&A\n")

documents = [
    Document(text="MLflow is an open source platform for the complete ML lifecycle. It provides experiment tracking, model registry, and deployment capabilities."),
    Document(text="MLflow Tracing captures the complete execution of GenAI applications, including LLM calls, retrieval steps, and tool usage."),
    Document(text="MLflow integrates with 30+ frameworks including OpenAI, LangChain, LlamaIndex, and more."),
    Document(text="MLflow supports collaborative development with experiment sharing, prompt management, and model versioning."),
]

# Create index (automatically traced)
index = VectorStoreIndex.from_documents(documents)

# Create query engine
query_engine = index.as_query_engine()

# Query the index (automatically traced)
response = query_engine.query("What tracing capabilities does MLflow have?")

print(f"Query: What tracing capabilities does MLflow have?")
print(f"\nAnswer: {response}")

print("\n✅ LlamaIndex execution fully traced!")
print("   - Document indexing")
print("   - Query embedding")
print("   - Retrieval")
print("   - Response synthesis")

### LlamaIndex Strengths

- ✅ **Indexing**: Advanced document indexing
- ✅ **Data loaders**: 100+ data connectors
- ✅ **RAG**: Specialized for retrieval
- ✅ **Query engines**: Multiple query strategies
- ✅ **Agents**: Data-aware agents
- ✅ **Performance**: Optimized for documents

---
## Step 6: Comparing Traces Across Frameworks

Let's run the same query using all three frameworks and compare.

In [ ]:
import time

query = "What is the main benefit of using MLflow for GenAI?"

results = []

print("\n🔬 Comparing frameworks...\n")

# Test 1: OpenAI
print("Testing OpenAI...")
start = time.time()
response1 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": query}],
    temperature=0.3,
    max_tokens=100
)
latency1 = time.time() - start
answer1 = response1.choices[0].message.content
tokens1 = response1.usage.total_tokens
results.append(("OpenAI", latency1, tokens1, len(answer1)))
print(f"  ✓ {latency1:.2f}s, {tokens1} tokens\n")

# Test 2: LangChain
print("Testing LangChain...")
start = time.time()
chain = ChatPromptTemplate.from_template("{query}") | \
        ChatOpenAI(model="gpt-4o-mini", temperature=0.3, max_tokens=100) | \
        StrOutputParser()
answer2 = chain.invoke({"query": query})
latency2 = time.time() - start
results.append(("LangChain", latency2, "N/A", len(answer2)))
print(f"  ✓ {latency2:.2f}s\n")

# Test 3: LlamaIndex
print("Testing LlamaIndex...")
start = time.time()
response3 = query_engine.query(query)
latency3 = time.time() - start
answer3 = str(response3)
results.append(("LlamaIndex", latency3, "N/A", len(answer3)))
print(f"  ✓ {latency3:.2f}s\n")

# Display comparison
print("="*60)
print("COMPARISON RESULTS")
print("="*60)
print(f"{'Framework':<15} {'Latency':<12} {'Tokens':<10} {'Length'}")
print("-"*60)
for framework, latency, tokens, length in results:
    print(f"{framework:<15} {latency:<12.2f} {str(tokens):<10} {length}")

print("\n💡 Observations:")
print("   - OpenAI: Lowest latency (direct API)")
print("   - LangChain: Slight overhead from abstractions")
print("   - LlamaIndex: Extra time for retrieval")
print("\n🔍 View all traces in MLflow UI to see the details!")

---
## Step 7: Choosing the Right Framework

### Decision Tree

```
Do you need document retrieval?
├─ Yes → Consider LlamaIndex
│   └─ Complex indexing strategies? → LlamaIndex
│   └─ Simple RAG? → LangChain or OpenAI
│
└─ No → Do you need agents?
    ├─ Yes → LangChain
    │   └─ Complex multi-tool workflows? → LangChain
    │   └─ Simple agent? → OpenAI + custom code
    │
    └─ No → OpenAI (for simplicity and performance)
```

### Real-World Recommendations

**Use OpenAI when:**
- Building a simple chatbot
- Need maximum performance
- Want full control
- Prototyping quickly

**Use LangChain when:**
- Building complex workflows
- Need agent capabilities
- Want reusable components
- Large team needs abstractions

**Use LlamaIndex when:**
- Document Q&A is primary use case
- Multiple data sources
- Advanced indexing needed
- Knowledge management focus

**Mix frameworks when:**
- You need strengths of multiple
- Different parts of application
- Gradual migration

---
## Step 8: Best Practices by Framework

### OpenAI Best Practices

```python
# DO:
✅ Use structured prompts
✅ Implement retry logic
✅ Handle rate limits
✅ Stream responses for UX
✅ Cache results when possible

# DON'T:
❌ Hardcode prompts
❌ Ignore error responses
❌ Skip cost tracking
❌ Use synchronous calls in production
```

### LangChain Best Practices

```python
# DO:
✅ Use LCEL for chains
✅ Leverage built-in components
✅ Test chains independently
✅ Use async for better performance
✅ Enable debug mode during development

# DON'T:
❌ Over-abstract simple use cases
❌ Ignore performance overhead
❌ Skip error handling in chains
❌ Use deprecated components
```

### LlamaIndex Best Practices

```python
# DO:
✅ Choose appropriate index type
✅ Chunk documents thoughtfully
✅ Use metadata for filtering
✅ Cache embeddings
✅ Monitor retrieval quality

# DON'T:
❌ Index without preprocessing
❌ Use default settings blindly
❌ Ignore index refresh strategies
❌ Skip evaluation of retrievals
```

---
## Summary

In this notebook, you learned:

1. ✅ Overview of MLflow's 30+ framework integrations
2. ✅ Working with OpenAI (direct API)
3. ✅ Working with LangChain (chains and workflows)
4. ✅ Working with LlamaIndex (document Q&A)
5. ✅ Comparing frameworks side-by-side
6. ✅ Choosing the right framework for your use case
7. ✅ Best practices for each framework

### Key Takeaways

- **All frameworks** are automatically traced by MLflow
- **Choose based on use case**, not hype
- **OpenAI** for simplicity and performance
- **LangChain** for complex workflows
- **LlamaIndex** for document-heavy applications
- **Mix frameworks** when it makes sense

### Framework Comparison Summary

| Aspect | OpenAI | LangChain | LlamaIndex |
|--------|--------|-----------|------------|
| **Best For** | Simple apps | Workflows | Doc Q&A |
| **Performance** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐⭐ |
| **Ease of Use** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐⭐ |
| **Flexibility** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐ |
| **RAG Support** | ⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ |
| **Agents** | ⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐ |

### What's Next?

**📓 Notebook 1.7: Complete RAG Application**

Put everything together:
- Build a production-ready RAG system
- Combine experiment tracking and tracing
- Implement caching and error handling
- Add monitoring and alerting
- Deploy with confidence!

### Additional Resources

- [MLflow LangChain Integration](https://mlflow.org/docs/latest/llms/langchain/index.html)
- [MLflow LlamaIndex Integration](https://mlflow.org/docs/latest/llms/llama-index/index.html)
- [Framework Examples](https://github.com/mlflow/mlflow/tree/master/examples)